In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


#model building
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import mean_squared_log_error,accuracy_score,roc_auc_score, r2_score,mean_squared_error

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train['gender']=np.where(train['gender']=='female',0,1)

In [3]:
train.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,0
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,0
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,0
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1


In [4]:
test.head()

,session_id,startTime,endTime,ProductList
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/


In [5]:
# check if prodcuts presetn in test and not in train and vice_versa

text_train=''
for t in train.ProductList:
    text_train+=t
class_train = list(set(re.findall(r'C[0-9]+',text_train)))

text_test=""
for t in test.ProductList:
    text_test+=t
class_test = list(set(re.findall(r'C[0-9]+',text_test)))

len(set(class_test)-set(class_train)),len(set(class_train)-set(class_test))

(23, 101)

In [6]:
global subSubCat_temp
df=train[['ProductList','gender']]
    
df=pd.concat([pd.Series(row['gender'], row['ProductList'].split(';'))              
                    for _, row in df.iterrows()]).reset_index()

df.columns=['all','gender']
temp=df.copy()
temp=temp['all'].str.split('/',expand=True)

temp.columns=['category','subCategory','subSubCategory','product','gender']
temp['gender']=df.gender

subSubCat_temp=temp.groupby(['subSubCategory','gender']).size().unstack(fill_value=0).reset_index()
subSubCat_temp.columns=['subSubCategory','female','male']
subSubCat_temp.head()


global subCat_temp
df=train[['ProductList','gender']]
    
df=pd.concat([pd.Series(row['gender'], row['ProductList'].split(';'))              
                    for _, row in df.iterrows()]).reset_index()

df.columns=['all','gender']
temp=df.copy()
temp=temp['all'].str.split('/',expand=True)

temp.columns=['category','subCategory','subSubCategory','product','gender']
temp['gender']=df.gender

subCat_temp=temp.groupby(['subCategory','gender']).size().unstack(fill_value=0).reset_index()
subCat_temp.columns=['subCategory','female','male']
subCat_temp.head()



global cat_temp
df=train[['ProductList','gender']]
    
df=pd.concat([pd.Series(row['gender'], row['ProductList'].split(';'))              
                    for _, row in df.iterrows()]).reset_index()

df.columns=['all','gender']
temp=df.copy()
temp=temp['all'].str.split('/',expand=True)

temp.columns=['category','subCategory','subSubCategory','product','gender']
temp['gender']=df.gender

cat_temp=temp.groupby(['category','gender']).size().unstack(fill_value=0).reset_index()
cat_temp.columns=['category','female','male']
cat_temp.head()

global prod_temp
df=train[['ProductList','gender']]
    
df=pd.concat([pd.Series(row['gender'], row['ProductList'].split(';'))              
                    for _, row in df.iterrows()]).reset_index()

df.columns=['all','gender']
temp=df.copy()
temp=temp['all'].str.split('/',expand=True)

temp.columns=['category','subCategory','subSubCategory','product','gender']
temp['gender']=df.gender

prod_temp=temp.groupby(['product','gender']).size().unstack(fill_value=0).reset_index()
prod_temp.columns=['product','female','male']
prod_temp.head()

,product,female,male
0,D00001,1,1
1,D00002,1,0
2,D00003,1,0
3,D00006,1,0
4,D00007,1,0


In [7]:
def catAll(x):
    prods = list(set(re.findall(r'A[0-9]+',x)))
    return prods

def subCatAll(x):
    prods = list(set(re.findall(r'B[0-9]+',x)))
    return prods

def subSubCatAll(x):
    prods = list(set(re.findall(r'C[0-9]+',x)))
    return prods

def productAll(x):
    prods = list(set(re.findall(r'D[0-9]+',x)))
    return prods


train['cat'] = train['ProductList'].apply(lambda x: catAll(x))
test['cat'] = test['ProductList'].apply(lambda x: catAll(x))

train['subCat'] = train['ProductList'].apply(lambda x: subCatAll(x))
test['subCat'] = test['ProductList'].apply(lambda x: subCatAll(x))

train['subSubCat'] = train['ProductList'].apply(lambda x: subSubCatAll(x))
test['subSubCat'] = test['ProductList'].apply(lambda x: subSubCatAll(x))

train['product'] = train['ProductList'].apply(lambda x: productAll(x))
test['product'] = test['ProductList'].apply(lambda x: productAll(x))

In [8]:
def productGenderMale(x):
    global prod_temp
    count=0
    for pro in x:
        if len(prod_temp[prod_temp['product']==str(pro)])>0:
            count += int(prod_temp[prod_temp['product']==str(pro)].male)
        else:
            count=np.nan
            break
    return count

def productGenderFemale(x):
    global prod_temp
    count=0
    for pro in x:
        if len(prod_temp[prod_temp['product']==str(pro)])>0:
            count += int(prod_temp[prod_temp['product']==str(pro)].female)
        else:
            count=np.nan
            break
    return count
        
    
train['productMale'] = train['product'].apply(lambda x: productGenderMale(x))
train['productFemale'] = train['product'].apply(lambda x: productGenderFemale(x))

test['productMale'] = test['product'].apply(lambda x: productGenderMale(x))
test['productFemale'] = test['product'].apply(lambda x: productGenderFemale(x))

# train.drop(['product'],axis=1,inplace=True)
# test.drop(['product'],axis=1,inplace=True)

In [9]:
def catGenderMale(x):
    global cat_temp
    count=0
    for pro in x:
        if len(cat_temp[cat_temp['category']==str(pro)])>0:
            count += int(cat_temp[cat_temp['category']==str(pro)].male)
        else:
            count=np.nan
            break
    return count

def catGenderFemale(x):
    global cat_temp
    count=0
    for pro in x:
        if len(cat_temp[cat_temp['category']==str(pro)])>0:
            count += int(cat_temp[cat_temp['category']==str(pro)].female)
        else:
            count=np.nan
            break
    return count
        
    
train['catMale'] = train['cat'].apply(lambda x: catGenderMale(x))
train['catFemale'] = train['cat'].apply(lambda x: catGenderFemale(x))

test['catMale'] = test['cat'].apply(lambda x: catGenderMale(x))
test['catFemale'] = test['cat'].apply(lambda x: catGenderFemale(x))

# train.drop(['cat'],axis=1,inplace=True)
# test.drop(['cat'],axis=1,inplace=True)

In [10]:
def subCatGenderMale(x):
    global subCat_temp
    count=0
    for pro in x:
        if len(subCat_temp[subCat_temp['subCategory']==str(pro)])>0:
            count += int(subCat_temp[subCat_temp['subCategory']==str(pro)].male)
        else:
            count=np.nan
            break
    return count

def subCatGenderFemale(x):
    global subCat_temp
    count=0
    for pro in x:
        if len(subCat_temp[subCat_temp['subCategory']==str(pro)])>0:
            count += int(subCat_temp[subCat_temp['subCategory']==str(pro)].female)
        else:
            count=np.nan
            break
    return count
        
    
train['subCatMale'] = train['subCat'].apply(lambda x: subCatGenderMale(x))
train['subCatFemale'] = train['subCat'].apply(lambda x: subCatGenderFemale(x))

test['subCatMale'] = test['subCat'].apply(lambda x: subCatGenderMale(x))
test['subCatFemale'] = test['subCat'].apply(lambda x: subCatGenderFemale(x))

# train.drop(['subCat'],axis=1,inplace=True)
# test.drop(['subCat'],axis=1,inplace=True)

In [11]:
def subSubCatGenderMale(x):
    global subSubCat_temp
    count=0
    for pro in x:
        if len(subSubCat_temp[subSubCat_temp['subSubCategory']==str(pro)])>0:
            count += int(subSubCat_temp[subSubCat_temp['subSubCategory']==str(pro)].male)
        else:
            count=np.nan
            break
    return count

def subSubCatGenderFemale(x):
    global subSubCat_temp
    count=0
    for pro in x:
        if len(subSubCat_temp[subSubCat_temp['subSubCategory']==str(pro)])>0:
            count += int(subSubCat_temp[subSubCat_temp['subSubCategory']==str(pro)].female)
        else:
            count= np.nan
            break
    return count
        
    
train['subSubCatMale'] = train['subSubCat'].apply(lambda x: subSubCatGenderMale(x))
train['subSubCatFemale'] = train['subSubCat'].apply(lambda x: subSubCatGenderFemale(x))

test['subSubCatMale'] = test['subSubCat'].apply(lambda x: subSubCatGenderMale(x))
test['subSubCatFemale'] = test['subSubCat'].apply(lambda x: subSubCatGenderFemale(x))

# train.drop(['subSubCat'],axis=1,inplace=True)
# test.drop(['subSubCat'],axis=1,inplace=True)

In [12]:
train['productMaleRatio'] = train['productMale']/(train['productMale']+train['productFemale'])
train['catMaleRatio'] = train['catMale']/(train['catMale']+train['catFemale'])
train['subCatMaleRatio'] = train['subCatMale']/(train['subCatMale']+train['subCatFemale'])
train['subSubCatMaleRatio'] = train['subSubCatMale']/(train['subSubCatMale']+train['subSubCatFemale'])

test['productMaleRatio'] = test['productMale']/(test['productMale']+test['productFemale'])
test['catMaleRatio'] = test['catMale']/(test['catMale']+test['catFemale'])
test['subCatMaleRatio'] = test['subCatMale']/(test['subCatMale']+test['subCatFemale'])
test['subSubCatMaleRatio'] = test['subSubCatMale']/(test['subSubCatMale']+test['subSubCatFemale'])

In [13]:
train['startTime']=pd.to_datetime(train['startTime'])
train['endTime']=pd.to_datetime(train['endTime'])
train['totalTime'] = (train['endTime']-train['startTime'])/np.timedelta64(1,'s')
train['dayName']=train['startTime'].dt.weekday_name
train['dayHour']=train['startTime'].dt.hour
train['totalTime']=train['totalTime'].abs()

test['startTime']=pd.to_datetime(test['startTime'])
test['endTime']=pd.to_datetime(test['endTime'])
test['totalTime'] = (test['endTime']-test['startTime'])/np.timedelta64(1,'s')
test['dayName']=test['startTime'].dt.weekday_name
test['dayHour']=test['startTime'].dt.hour
test['totalTime']=test['totalTime'].abs()

In [14]:
q1=train['totalTime'].quantile(0.25)
q2=train['totalTime'].quantile(0.75)
iqr=q2-q1
upper_bound=q2+1.5*(iqr)
lowerbound=q1-1.5*(iqr)

train['totalTime']=np.where(train['totalTime']>upper_bound,upper_bound,train['totalTime'])
test['totalTime']=np.where(test['totalTime']>upper_bound,upper_bound,test['totalTime'])

In [15]:
# train[train['dayName']=='Friday'].groupby(['gender'])['dayHour'].plot(kind='hist',bins=np.arange(0,24,1))
# plt.show()

In [16]:
# t = train.groupby(['dayName','gender']).size().unstack()
# t.columns=['female','male']
# t['female_per']=t['female']/t.sum(axis=1)
# t['male_per']=t['male']/t.sum(axis=1)
# t

# train.groupby(['gender'])['dayHour'].plot(kind='hist')
# plt.show()

In [17]:
# #CHECKING CATEGORY WISE DISTRIBUTION
# text=''
# for t in train.ProductList:
#     text+=t
# category = list(set(re.findall(r'A[0-9]+',text)))

# df = train.copy()
# for name in category:
#     df[name]=np.nan
#     df[name]=np.nan

# def categoryVerify(x,name):
#     check=list(set(re.findall(r'A[0-9]+',x)))
#     if name in check:
#         return 1
#     else:
#         return 0

# for name in category:
#     df[name]=df['ProductList'].apply(lambda x: categoryVerify(x,name))

    
# cols=category
# cols.append('gender')
# df=df[cols]
# t=df.groupby(['gender']).agg('sum')
# t.T

In [18]:
# #CHECKING SUBCATEGORY WISE DISTRIBUTION
# text=''
# for t in train.ProductList:
#     text+=t
# subCategory = list(set(re.findall(r'B[0-9]+',text)))

# df = train.copy()
# for name in subCategory:
#     df[name]=np.nan
#     df[name]=np.nan

# def subCategoryVerify(x,name):
#     check=list(set(re.findall(r'B[0-9]+',x)))
#     if name in check:
#         return 1
#     else:
#         return 0

# for name in subCategory:
#     df[name]=df['ProductList'].apply(lambda x: subCategoryVerify(x,name))

    
# cols=subCategory
# cols.append('gender')
# df=df[cols]
# t=df.groupby(['gender']).agg('sum')
# t.T

In [19]:
# #CHECKING subSubCategory WISE DISTRIBUTION
# text=''
# for t in train.ProductList:
#     text+=t
# subSubCategory = list(set(re.findall(r'C[0-9]+',text)))

# df = train.copy()
# for name in subSubCategory:
#     df[name]=np.nan
#     df[name]=np.nan

# def subSubCategoryVerify(x,name):
#     check=list(set(re.findall(r'C[0-9]+',x)))
#     if name in check:
#         return 1
#     else:
#         return 0

# for name in subSubCategory:
#     df[name]=df['ProductList'].apply(lambda x: subSubCategoryVerify(x,name))

    
# cols=subSubCategory
# cols.append('gender')
# df=df[cols]
# t=df.groupby(['gender']).agg('sum')
# t.T

In [20]:
text=''
for t in train.ProductList:
    text+=t
category = list(set(re.findall(r'A[0-9]+',text)))
subCategory = list(set(re.findall(r'B[0-9]+',text)))
subSubCategory = list(set(re.findall(r'C[0-9]+',text)))
product = list(set(re.findall(r'D[0-9]+',text)))


def catCount(x):
    n=len(re.findall(r'A[0-9]+',x))
    return n
def subCatCount(x):
    n=len(re.findall(r'B[0-9]+',x))
    return n
def subSubCatCount(x):
    n=len(re.findall(r'C[0-9]+',x))
    return n
def productCount(x):
    n=len(re.findall(r'D[0-9]+',x))
    return n

# train['numCategory']=train['ProductList'].apply(lambda x: catCount(x))
# train['numSubCategory']=train['ProductList'].apply(lambda x: subCatCount(x))
# train['numSubSubCategory']=train['ProductList'].apply(lambda x: subSubCatCount(x))
train['productCount']=train['ProductList'].apply(lambda x: productCount(x))

# test['numCategory']=test['ProductList'].apply(lambda x: catCount(x))
# test['numSubCategory']=test['ProductList'].apply(lambda x: subCatCount(x))
# test['numSubSubCategory']=test['ProductList'].apply(lambda x: subSubCatCount(x))
test['productCount']=test['ProductList'].apply(lambda x: productCount(x))

train['productTime'] = train['totalTime']/train['productCount']
test['productTime'] = test['totalTime']/test['productCount']

In [21]:
for name in category:
    train[name]=np.nan
    test[name]=np.nan

def catVerify(x,name):
    products=list(re.findall(r'A[0-9]+',x))
    if name in products:
        return 1
    else:
        return 0
    
for name in category:
    train[name]=train['ProductList'].apply(lambda x: catVerify(x,name))
    test[name]=test['ProductList'].apply(lambda x: catVerify(x,name))

#_____________________________________________________________________________
for name in subCategory:
    train[name]=np.nan
    test[name]=np.nan

def subCatVerify(x,name):
    products=list(re.findall(r'B[0-9]+',x))
    if name in products:
        return 1
    else:
        return 0
    
for name in subCategory:
    train[name]=train['ProductList'].apply(lambda x: subCatVerify(x,name))
    test[name]=test['ProductList'].apply(lambda x: subCatVerify(x,name))
    
    
#_____________________________________________________________________________
for name in subSubCategory:
    train[name]=np.nan
    test[name]=np.nan

def subSubCatVerify(x,name):
    products=list(re.findall(r'C[0-9]+',x))
    if name in products:
        return 1
    else:
        return 0
    
for name in subSubCategory:
    train[name]=train['ProductList'].apply(lambda x: subSubCatVerify(x,name))
    test[name]=test['ProductList'].apply(lambda x: subSubCatVerify(x,name))

In [22]:
train=pd.get_dummies(train,columns=['dayName'])
test=pd.get_dummies(test,columns=['dayName'])

# test["productMaleRatio"] = test['productMaleRatio'].fillna(test.groupby(['catMaleRatio','subCatMaleRatio','subSubCatMaleRatio'])\
#                                                            ['productMaleRatio'].transform('mean'))

# test.fillna(test.mean(),inplace=True)

In [68]:
test['total_malefemale']=test['productMale']+test['productFemale']

In [83]:
test1=test[~test['productMaleRatio'].isna()]
test2=test[~test['session_id'].isin(test1.session_id)]
test1.shape,test2.shape,test.shape

((1521, 488), (2979, 488), (4500, 488))

In [87]:
# X_1=train[[col for col in train.columns if col not in (['session_id','startTime','endTime','ProductList','gender',
#                                                       'cat','subCat','subSubCat','product','productMaleRatio',
#                                                       'productMale','productFemale',
#                                                       'catMale','catFemale',
#                                                       'subCatMale','subCatFemale',
#                                                       'subSubCatMale','subSubCatFemale',
#                                                         'totalTime','dayHour','productCount','productTime',
#                                                         'dayName_Friday','dayName_Monday','dayName_Saturday','dayName_Sunday',
#                                                         'dayName_Thursday','dayName_Tuesday','dayName_Wednesday'
#                                                      ])]]

X_1=train[['productMaleRatio']]
# text = " ".join(X_1.columns)
# extra_col = list(set(re.findall(r'[C][0-9]+',text)))
# X_1 = X_1[[col for col in X_1.columns if col not in extra_col]]
y_1=train.gender

test_1=test1.copy()
test_1=test_1[X_1.columns]

skf=StratifiedKFold(n_splits=10,shuffle=True)
cv_score = []

lgb=lightgbm.LGBMClassifier(boosting_type='gbdt', 
                      objective='binary',
                      num_iteration=3000,
                      learning_rate=0.01,
                      max_depth=5,
#                       colsample_bytree=0.9,
                      reg_alpha=0.0,
                      reg_lambda=0.0)
rfc=RandomForestClassifier(n_estimators=500)
lr=LogisticRegression(C=2)
xgb = XGBClassifier(n_estimators=1000,
                    max_depth=5,
                    learning_rate=0.1)

clfs=[lgb,xgb]

cols= ['lgb','xgb']
final1=pd.DataFrame(columns=cols)
i=0
for clf_ in clfs:

    pred_test =np.zeros((len(test_1),1))

    for train_index,test_index in skf.split(X_1,y_1):
        x_train,x_val = X_1.iloc[train_index],X_1.iloc[test_index]
        y_train,y_val = y_1.iloc[train_index],y_1.iloc[test_index]
        clf = clf_
        clf.fit(x_train,y_train)
        score = accuracy_score(y_val,clf.predict(x_val))
        cv_score.append(score)
        print(round(score,3),end=",")

        pred_test += clf.predict_proba(test_1)[:,1].reshape(-1,1)
    pred_test=pred_test/10 
    final1[cols[i]]=pred_test[:,0]
    i+=1
    print("\nmean_score: ",round(np.mean(cv_score),3))

0.974,0.982,0.98,0.971,0.973,0.978,0.976,0.96,0.978,0.977,
mean_score:  0.975
0.977,0.969,0.977,0.971,0.971,0.972,0.973,0.973,0.978,0.98,
mean_score:  0.975


In [101]:
X_2=train[[col for col in train.columns if col not in (['session_id','startTime','endTime','ProductList','gender',
                                                      'cat','subCat','subSubCat','product',
                                                      'productMale','productFemale','productMaleRatio'

                                                     ])]]

# X_2=train[['catMaleRatio','subCatMaleRatio','subSubCatMaleRatio']]
# text = " ".join(X_2.columns)
# extra_col = list(set(re.findall(r'[C][0-9]+',text)))
# X_2 = X_2[[col for col in X_2.columns if col not in extra_col]]
y_2=train.gender

test_2=test2.copy()
test_2=test_2[X_2.columns]


skf=StratifiedKFold(n_splits=10,shuffle=True)
cv_score = []

lgb=lightgbm.LGBMClassifier(boosting_type='gbdt', 
                      objective='binary',
                      num_iteration=3000,
                      learning_rate=0.01,
                      max_depth=5,
#                       colsample_bytree=0.9,
                      reg_alpha=0.0,
                      reg_lambda=0.0)
rfc=RandomForestClassifier(n_estimators=500)
lr=LogisticRegression(C=2)
xgb = XGBClassifier(n_estimators=1000,
                    max_depth=5,
                    learning_rate=0.1)

clfs=[lgb,xgb]

cols= ['lgb','xgb']
final2=pd.DataFrame(columns=cols)
i=0
for clf_ in clfs:

    pred_test =np.zeros((len(test_2),1))

    for train_index,test_index in skf.split(X_2,y_2):
        x_train,x_val = X_2.iloc[train_index],X_2.iloc[test_index]
        y_train,y_val = y_2.iloc[train_index],y_2.iloc[test_index]
        clf = clf_
        clf.fit(x_train,y_train)
        score = accuracy_score(y_val,clf.predict(x_val))
        cv_score.append(score)
        print(round(score,3),end=",")

        pred_test += clf.predict_proba(test_2)[:,1].reshape(-1,1)
    pred_test=pred_test/10 
    final2[cols[i]]=pred_test[:,0]
    i+=1
    print("\nmean_score: ",round(np.mean(cv_score),3))

0.886,0.871,0.87,0.883,0.867,0.883,0.878,0.867,0.873,0.883,
mean_score:  0.876
0.874,0.874,0.887,0.86,0.874,0.866,0.87,0.879,0.872,0.886,
mean_score:  0.875


In [102]:
final1['session_id']=test1.session_id.values
final2['session_id']=test2.session_id.values

final=pd.concat([final1,final2],ignore_index=True)
final['final_score']=(final['xgb']+final['lgb'])/2

temp=test.copy()
temp=pd.merge(temp,final,on='session_id',how='left')
temp.index=temp.session_id
temp=temp[['final_score']]

temp['gender']=np.where(temp['final_score']>=0.5,'male','female')
temp=temp[['gender']]
temp.to_csv('try8.csv')
temp.head()

,gender
session_id,
u12112,female
u19725,female
u11795,female
u22639,male
u18034,female


In [103]:
# final['overall']=final.sum(axis=1)/2
# temp=test.copy()
# temp['prob']=final.overall
# temp['gender']=np.where(temp['prob']>=0.5,'male','female')
# temp.index=temp.session_id
# temp=temp[['gender']]
# temp.to_csv('try8.csv')
# temp.head()

In [104]:
temp['gender'].value_counts(normalize=True)

female    0.825556
male      0.174444
Name: gender, dtype: float64

In [99]:
train['gender'].value_counts(normalize=True)

0    0.78019
1    0.21981
Name: gender, dtype: float64